<a href="https://colab.research.google.com/github/JeremyAnretar/Business-Use-Cases/blob/main/customer_base_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('Wholesale customers data.csv')
df.head(5)

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185


In [ ]:
#Let's decribe our dataframe
df.describe()

,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen,CA_Global
count,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000
mean,12000.297727,5796.265909,7951.277273,3071.931818,2881.493182,1524.870455,33226.136364
std,12647.328865,7380.377175,9503.162829,4854.673333,4767.854448,2820.105937,26356.301730
min,3.000000,55.000000,3.000000,25.000000,3.000000,3.000000,904.000000
25%,3127.750000,1533.000000,2153.000000,742.250000,256.750000,408.250000,17448.750000
50%,8504.000000,3627.000000,4755.500000,1526.000000,816.500000,965.500000,27492.000000
75%,16933.750000,7190.250000,10655.750000,3554.250000,3922.000000,1820.250000,41307.500000
max,112151.000000,73498.000000,92780.000000,60869.000000,40827.000000,47943.000000,199891.000000


In [ ]:
#We start with renaming
df['Channel'] = df['Channel'].map({1:'Hotel/Restaurant/Cafe', 2:'Retail'})
df['Region'] = df['Region'].map({1:'Lisbon', 2:'Porto', 3:'Other'})
df.head(5)

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,Retail,Other,12669,9656,7561,214,2674,1338
1,Retail,Other,7057,9810,9568,1762,3293,1776
2,Retail,Other,6353,8808,7684,2405,3516,7844
3,Hotel/Restaurant/Cafe,Other,13265,1196,4221,6404,507,1788
4,Retail,Other,22615,5410,7198,3915,1777,5185


In [ ]:
# ------------------------------------------------------------
# Project: Keep top customers per (Channel, Region) under constraints
# Author: sharing the approach I used on LinkedIn 🙂
#
# TL;DR (what I told LinkedIn):
# - I needed to keep a fixed number of customers per (Channel, Region):
#     • Lisbon & Porto -> keep 10 customers
#     • Other -> keep 20 customers
# - Objective: maximize total revenue (CA_Global = sum of Fresh/Milk/Grocery/Frozen/Detergents_Paper/Delicassen)
# - Constraint (per group): selected rows must cover at least 50% of the group’s Frozen
# - Strategy:
#     1) Start with the top-k by CA_Global (maximize revenue first)
#     2) If Frozen coverage is < 50%, swap in high-Frozen candidates (greedy)
#     3) If 50% is impossible with k rows, take the top-k by Frozen (best possible coverage) and flag the group as infeasible
#
# Why this works for business:
# - It respects headcount/operational limits (fixed quota).
# - It preserves revenue as much as possible.
# - It enforces a margin-critical constraint (Frozen coverage) where we make the best margin.
# ------------------------------------------------------------

import pandas as pd

# --- 0) Data assumptions ------------------------------------------------------
# You already have a DataFrame `df` with:
#   ['Channel','Region','Fresh','Milk','Grocery','Frozen','Detergents_Paper','Delicassen']
# (If your column names differ, remap them here.)

# --- 1) Normalize the 'Region' labels ----------------------------------------
# Real life data is… real life. "Lisboa", stray spaces, weird casing… normalize it.
df["Region"] = (
    df["Region"]
    .astype(str)
    .str.strip()                  # kill leading/trailing spaces
    .str.title()                  # "lisbon" -> "Lisbon"
    .replace({"Lisboa": "Lisbon", "Oporto": "Porto"})  # common variants
)

# If you're debugging weird quotas later, uncomment this:
# print(sorted(df["Region"].unique()))

# --- 2) Build the revenue we want to maximize --------------------------------
revenue_cols = ["Fresh","Milk","Grocery","Frozen","Detergents_Paper","Delicassen"]

# Keep it explicit: CA_Global is just the sum of all spend categories
df["CA_Global"] = df[revenue_cols].sum(axis=1)

# --- 3) Quota rule: simple and explicit --------------------------------------
# On LinkedIn, I always explain the rule in plain English first:
# - If Region ∈ {Lisbon, Porto} → keep 10 rows
# - Else (Other) → keep 20 rows
PORTO_LISBON = {"Porto", "Lisbon"}

def quota_for_region(region: str) -> int:
    return 10 if region in PORTO_LISBON else 20

# --- 4) Core selection logic per (Channel, Region) ---------------------------
def select_group(group: pd.DataFrame) -> pd.DataFrame:
    """
    Business goal (1 group):
      - Keep exactly k rows (k depends on Region).
      - Maximize CA_Global (we start with top-k by revenue).
      - Enforce 'Frozen coverage' >= 50% of the group's Frozen.
      - If impossible to reach 50% with k rows, we keep top-k by Frozen (best effort)
        and mark the group as infeasible.

    Why a greedy swap (plain English):
      - It’s fast, readable, and aligns with how a human would “fix” a selection:
        start from the best revenue table, then swap in rows that improve Frozen
        until we satisfy the constraint.
    """
    # Meta (all rows share the same channel & region)
    channel = group["Channel"].iloc[0]
    region  = group["Region"].iloc[0]
    k = quota_for_region(region)

    # If the group has fewer rows than the quota, we keep them all.
    # Operationally, that happens in small segments — and it auto-covers 100% of Frozen.
    if len(group) <= k:
        sel = group.copy()
        frozen_total = group["Frozen"].sum()
        sel["__frozen_threshold__"] = 0.5 * frozen_total
        sel["__frozen_selected__"] = sel["Frozen"].sum()
        sel["__frozen_coverage__"] = (sel["__frozen_selected__"] / frozen_total) if frozen_total > 0 else 0.0
        sel["__required_quota__"] = k
        sel["__feasible__"] = True
        return sel

    # Compute the per-group Frozen requirement once (avoid magic numbers later)
    frozen_total = group["Frozen"].sum()
    frozen_threshold = 0.5 * frozen_total

    # Quick feasibility check:
    # Even if I take the top-k rows by Frozen, do I reach 50%?
    # If not, it's mathematically impossible with k rows → pick best possible Frozen and flag infeasible.
    top_k_by_frozen = group.nlargest(k, "Frozen")
    if top_k_by_frozen["Frozen"].sum() < frozen_threshold:
        sel = top_k_by_frozen.copy()
        sel["__frozen_threshold__"] = frozen_threshold
        sel["__frozen_selected__"] = sel["Frozen"].sum()
        sel["__frozen_coverage__"] = (sel["__frozen_selected__"] / frozen_total) if frozen_total > 0 else 0.0
        sel["__required_quota__"] = k
        sel["__feasible__"] = False
        # Sanity check: enforce we really selected k rows
        assert len(sel) == k, f"Selected rows != k for group {(channel, region)}"
        return sel

    # If feasible, start from what maximizes revenue:
    sel = group.nlargest(k, "CA_Global").copy()

    # If we already meet the Frozen threshold, amazing — we’re done.
    if sel["Frozen"].sum() >= frozen_threshold:
        sel["__frozen_threshold__"] = frozen_threshold
        sel["__frozen_selected__"] = sel["Frozen"].sum()
        sel["__frozen_coverage__"] = (sel["__frozen_selected__"] / frozen_total) if frozen_total > 0 else 0.0
        sel["__required_quota__"] = k
        sel["__feasible__"] = True
        assert len(sel) == k, f"Selected rows != k for group {(channel, region)}"
        return sel

    # Otherwise, we need to boost Frozen using a human-ish greedy swap:
    # - Look at candidates not selected yet, sorted by Frozen (biggest wins first)
    selected_idx = set(sel.index)
    candidates = group.loc[~group.index.isin(selected_idx)].sort_values("Frozen", ascending=False).copy()

    for idx, cand in candidates.iterrows():
        # Identify the 'worst' selected row from a Frozen perspective
        worst_idx = sel["Frozen"].idxmin()

        # If the candidate isn't better on Frozen than our worst, we can't improve further → stop.
        if cand["Frozen"] <= sel.at[worst_idx, "Frozen"]:
            break

        # Swap: remove the worst, add the candidate
        sel = sel.drop(index=worst_idx)
        sel = pd.concat([sel, pd.DataFrame([cand])], axis=0)

        # Check if we now meet the constraint
        if sel["Frozen"].sum() >= frozen_threshold:
            break

    # One last check — if the initial feasibility test passed, we *should* be good now.
    feasible = sel["Frozen"].sum() >= frozen_threshold

    sel["__frozen_threshold__"] = frozen_threshold
    sel["__frozen_selected__"] = sel["Frozen"].sum()
    sel["__frozen_coverage__"] = (sel["__frozen_selected__"] / frozen_total) if frozen_total > 0 else 0.0
    sel["__required_quota__"] = k
    sel["__feasible__"] = bool(feasible)
    assert len(sel) == k, f"Selected rows != k for group {(channel, region)}"
    return sel

# --- 5) Apply per (Channel, Region) ------------------------------------------
selected_parts = []
for (channel, region), grp in df.groupby(["Channel", "Region"], dropna=False):
    # Keep the function call simple — it reads like English
    chosen = select_group(grp)
    selected_parts.append(chosen)

selected_df = pd.concat(selected_parts, axis=0, ignore_index=True)

# --- 6) Business-friendly summary (what I show on LinkedIn) -------------------
summary = (
    selected_df
    .groupby(["Channel","Region"], dropna=False, as_index=False)
    .agg(
        rows_selected=("Frozen", "size"),
        frozen_selected=("Frozen", "sum"),
        ca_global_selected=("CA_Global", "sum"),
        feasible=("**dummy**" if "__feasible__" not in selected_df.columns else "__feasible__", "min"),
        frozen_threshold=("__frozen_threshold__", "max"),
        frozen_coverage=("**dummy**" if "__frozen_coverage__" not in selected_df.columns else "__frozen_coverage__", "max"),
        required_quota=("__required_quota__", "max"),
    )
    .sort_values(["Channel", "Region"])
)

# Clean up column names if the **dummy** trick isn’t needed
# (It’s here just in case someone drops internal cols earlier.)
if "**dummy**" in summary.columns:
    summary = summary.drop(columns=["**dummy**"], errors="ignore")

print("Selection done.")
print("\n=== Summary per (Channel, Region) ===")
display(summary)

print("\n=== First selected rows (for a quick glance) ===")
display(
    selected_df
    .sort_values(["Channel","Region","CA_Global"], ascending=[True, True, False])
    .head(20)
)

# --- 7)
# What I liked about this approach:
# - It balances a hard operational constraint (quota) with a margin-driven constraint (Frozen >= 50%),
#   while still maximizing revenue where possible.
# - The code stays readable: if you skim it, you can guess the intent without reverse-engineering.
# - If I needed a mathematically guaranteed optimum, I'd switch to an ILP with PuLP/OR-Tools.
#   Here, the greedy is more than enough (and way simpler to maintain).


Selection done.

=== Summary per (Channel, Region) ===


,Channel,Region,rows_selected,frozen_selected,ca_global_selected,feasible,frozen_threshold,frozen_coverage,required_quota
0,Hotel/Restaurant/Cafe,Lisbon,10,87392,356635,False,92256.0,0.473639,10
1,Hotel/Restaurant/Cafe,Other,20,307247,1306816,False,385803.0,0.398192,20
2,Hotel/Restaurant/Cafe,Porto,10,86062,415826,True,80430.5,0.535008,10
3,Retail,Lisbon,10,26019,615766,True,23257.0,0.559380,10
4,Retail,Other,20,78621,1319619,False,79443.0,0.494826,20
5,Retail,Porto,10,22953,493762,True,14635.5,0.784155,10



=== First selected rows (for a quick glance) ===


,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen,CA_Global,__frozen_threshold__,__frozen_selected__,__frozen_coverage__,__required_quota__,__feasible__
7,Hotel/Restaurant/Cafe,Lisbon,56083,4563,2124,6422,730,3321,73243,92256.0,87392,0.473639,10,False
0,Hotel/Restaurant/Cafe,Lisbon,30624,7209,4897,18711,763,2876,65080,92256.0,87392,0.473639,10,False
2,Hotel/Restaurant/Cafe,Lisbon,5909,23527,13699,10155,830,3636,57756,92256.0,87392,0.473639,10,False
1,Hotel/Restaurant/Cafe,Lisbon,22096,3575,7041,11422,343,2564,47041,92256.0,87392,0.473639,10,False
3,Hotel/Restaurant/Cafe,Lisbon,6623,1860,4740,7683,205,1693,22804,92256.0,87392,0.473639,10,False
5,Hotel/Restaurant/Cafe,Lisbon,8656,2746,2501,6845,694,980,22422,92256.0,87392,0.473639,10,False
8,Hotel/Restaurant/Cafe,Lisbon,8040,3795,2070,6340,918,291,21454,92256.0,87392,0.473639,10,False
6,Hotel/Restaurant/Cafe,Lisbon,7858,1110,1094,6818,49,287,17216,92256.0,87392,0.473639,10,False
4,Hotel/Restaurant/Cafe,Lisbon,5041,1115,2856,7496,256,375,17139,92256.0,87392,0.473639,10,False
9,Hotel/Restaurant/Cafe,Lisbon,4720,1032,975,5500,197,56,12480,92256.0,87392,0.473639,10,False
